<a href="https://colab.research.google.com/github/DinhVinh2404/Books_Agent/blob/main/Book_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q langchain==0.3.7 langchain-community==0.3.5 sqlalchemy==2.0.35 ipython-sql

In [2]:
from google.colab import userdata
import os
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

**CREAT TABLES**

In [3]:
import sqlite3

db_path = "/content/books.db"
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
  CREATE TABLE IF NOT EXISTS Books (
    book_id INTEGER PRIMARY KEY,
    title TEXT,
    author TEXT,
    price REAL,
    stock INTEGER,
    category TEXT
  )
''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS Orders (
    order_id INTEGER PRIMARY KEY,
    customer_name TEXT,
    phone TEXT,
    address TEXT,
    book_id INTEGER,
    quantity INTEGER,
    status TEXT,
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
  )
''')

cursor.execute("INSERT INTO Books VALUES (1, 'Chatbot', 'Vinh', 24.04, 1, 'Test')")
cursor.execute("INSERT INTO Books VALUES (2, 'User manual', 'Vinh2', 2003, 5, 'Manual')")
conn.commit()
cursor.close()
conn.close()



In [4]:
%%capture --no-stderr
%pip install -q langgraph==0.3.10
%pip install -q langchain-google-genai
%pip install -q langgraph

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.tools import Tool

In [ ]:
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [7]:
def oder(customer_name, phone, address, book_id, quantity, status):
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()

  cursor.execute("SELECT * FROM Books WHERE book_id = ?", (book_id))
  book = cursor.fetchone()

  if book is None:
    return f"Book with ID {book_id} not found"
    cursor.close()
  conn.close()

  if book[4] < quantity:
    return f"Not enough stock for book with ID {book_id}"
    cursor.close()
  conn.close()

  cursor.execute("INSERT INTO Orders VALUES (NULL,?,?,?,?,?,?)", (customer_name, phone, address, book_id, quantity, status))
  conn.commit()
  cursor.close()
  conn.close()
  return f"Order placed successfully."
order_tools = Tool(name="oder", func=oder, description="Place an order into the Orders table. Provide customer_name, phone, address, book_id, quantity, status.",
                   args_schema={
                       "customer_name": str,
                       "phone": str,
                       "address": str,
                       "book_id": int,
                       "quantity": int,
                       "status": str
                   }
)

In [8]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 2 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the books table in database. Can make any changes to the orders table in database

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)
toolkit = SQLDatabaseToolkit(db=db, llm=model)
tools = toolkit.get_tools()
tools.append(order_tools)
agent_executor = create_react_agent(model=model, tools=tools, prompt=system_message)

In [8]:
import time

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Is there a book called Chatbot?")]}
):
    time.sleep(5) # Run sleep to prevent execeeding rate limits 60 RPM
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_list_tables', 'arguments': '{"tool_input": ""}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--50dac38f-694d-4085-897c-0f9c2e5e2370-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'afb2e082-122b-49a2-9a83-395e41f9e453', 'type': 'tool_call'}], usage_metadata={'input_tokens': 622, 'output_tokens': 111, 'total_tokens': 733, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 92}})]}}
----
{'tools': {'messages': [ToolMessage(content='Books, Orders', name='sql_db_list_tables', id='824bc42d-2e75-485e-b574-e2ed9f40c4d0', tool_call_id='afb2e082-122b-49a2-9a83-395e41f9e453')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_schema', 'arguments': '{"table_n

In [12]:
import time
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="I want to order 1 copy of Chatbot. My name is VinhDV, phone 012345678, address Hanoi")]}
):
    time.sleep(5)
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_list_tables', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--05647b62-f15a-4755-b7a0-afcc35b77eed-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': '5c964ba1-6830-4f99-ac52-eeb77525c8d9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 703, 'output_tokens': 71, 'total_tokens': 774, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 57}})]}}
----
{'tools': {'messages': [ToolMessage(content='Books, Orders', name='sql_db_list_tables', id='a23c283c-67f7-430b-b6d7-681a6c5105c9', tool_call_id='5c964ba1-6830-4f99-ac52-eeb77525c8d9')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_schema', 'arguments': '{"table_names": "Books"}'}}, response_meta

In [24]:
print("DB tables:", db.get_table_names())

DB tables: ['Books', 'Orders']


In [13]:
print(db.table_info)


CREATE TABLE "Books" (
	book_id INTEGER, 
	title TEXT, 
	author TEXT, 
	price REAL, 
	stock INTEGER, 
	category TEXT, 
	PRIMARY KEY (book_id)
)

/*
3 rows from Books table:
book_id	title	author	price	stock	category
1	Chatbot	Vinh	24.04	1	Test
2	User manual	Vinh2	2003.0	5	Manual
*/


CREATE TABLE "Orders" (
	order_id INTEGER, 
	customer_name TEXT, 
	phone TEXT, 
	address TEXT, 
	book_id INTEGER, 
	quantity INTEGER, 
	status TEXT, 
	PRIMARY KEY (order_id), 
	FOREIGN KEY(book_id) REFERENCES "Books" (book_id)
)

/*
3 rows from Orders table:
order_id	customer_name	phone	address	book_id	quantity	status
1	VinhDV	012345678	Hanoi	1	1	Pending
*/


In [14]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT * FROM Orders")
orders = cursor.fetchall()

for order in orders:
    print(order)

cursor.close()
conn.close()

(1, 'VinhDV', '012345678', 'Hanoi', 1, 1, 'Pending')
